In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import KarateClub
from torch_geometric.nn import GCNConv
from sklearn.metrics import accuracy_score, recall_score

In [2]:
# KROK 1: Przygotowanie danych
dataset = KarateClub()
data = dataset[0]

print(f"Liczba węzłów: {data.num_nodes}")
print(f"Liczba krawędzi: {data.num_edges}")
print(f"Liczba cech: {data.num_node_features}")

Liczba węzłów: 34
Liczba krawędzi: 156
Liczba cech: 34


In [3]:
# KROK 2: Definicja modelu
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GNN()

In [5]:
# KROK 3: Konfiguracja
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
# KROK 4: Trening
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss = train()

In [7]:
# KROK 5: Ewaluacja
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)

# KROK 6: Metryki
y_true = data.y[~data.train_mask].numpy()
y_pred = pred[~data.train_mask].numpy()

print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred, average='macro'):.4f}")

Accuracy: 0.8000
Recall: 0.8049
